In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Building a Spam Classifier with Naive Bayes

First of all getting our .txt files ready

In [ ]:
train_labels = pd.read_csv('/content/drive/MyDrive/train-labels.txt',header=None)
train_features= pd.read_csv('/content/drive/MyDrive/train-features.txt',delimiter=" ",header=None)



test_labels= pd.read_csv('/content/drive/MyDrive/test-labels.txt',header=None)
test_features= pd.read_csv('/content/drive/MyDrive/test-features.txt',delimiter=" ",header=None)


train_labels.rename(columns={0: 'Label'}, inplace=True)
#train_labels.head()

test_labels.rename(columns={0: 'Label'}, inplace=True)
#test_labels.head()

train_features.info()
train_labels.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Columns: 2500 entries, 0 to 2499
dtypes: int64(2500)
memory usage: 13.4 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Label   700 non-null    int64
dtypes: int64(1)
memory usage: 5.6 KB


# Question 1: Balanced or Skewed?

In [ ]:
count_of_zeros = (train_labels["Label"] == 0).sum()

count_of_ones = (train_labels["Label"] == 1).sum()

print("Ratio of label=1/label=0 is equal to:", count_of_zeros/count_of_ones)
print("The training set is balanced since there are equal amounts of label=0 and label=1")

Ratio of label=1/label=0 is equal to: 1.0
The training set is balanced since there are equal amounts of label=0 and label=1


# Question 2: Amount of Parameters?

For each word (there is 2500) we should find the probability of that word appearing in the email (given it is spam or not). So we need 2*2500 = 5000 parameters

Also, πy=1 estimates the probability that any particular email will be a spam email. This adds +1

# Building the Classifier

I created 2 dataframes and divided the data into spam and non-spam

In [ ]:
trainDF = pd.concat([train_features, train_labels],axis=1)
trainDF.head()

trainLabel1 = trainDF[trainDF["Label"] == 1]


trainLabel0 = trainDF[trainDF["Label"] == 0]
trainLabel0.describe()

,0,1,2,3,4,5,6,7,8,9,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,Label
count,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,...,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.0,350.0,350.0
mean,0.917143,0.720000,0.348571,3.228571,0.165714,0.402857,0.200000,2.588571,0.531429,0.328571,...,0.031429,0.022857,0.028571,0.028571,0.011429,0.017143,0.020000,0.0,0.0,0.0
std,1.735024,1.396434,0.956765,4.982984,0.843632,0.896023,0.876304,5.818347,1.031251,0.875018,...,0.388422,0.184011,0.249437,0.260671,0.106444,0.129989,0.159332,0.0,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
50%,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
75%,1.000000,1.000000,0.000000,4.000000,0.000000,0.000000,0.000000,2.750000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
max,14.000000,10.000000,10.000000,45.000000,13.000000,6.000000,14.000000,51.000000,8.000000,8.000000,...,7.000000,2.000000,3.000000,4.000000,1.000000,1.000000,2.000000,0.0,0.0,0.0


Since the dataframe stored the amount of times the word appeared in the mail. I created a temporary dataframe that stores if the word is inside the mail or not given that it is a non-spam

In [ ]:
trainLabel0_doesWordExist = trainLabel0
columns = 0
while columns <= 2499:
  rows = 0
  while rows < 350:
    if trainLabel0_doesWordExist.iloc[rows][columns] > 1:
      trainLabel0_doesWordExist.iloc[rows][columns] = 1
    rows +=1
  columns += 1

trainLabel0_doesWordExist.describe()

,0,1,2,3,4,5,6,7,8,9,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,Label
count,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,...,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.0,350.0,350.0
mean,0.388571,0.322857,0.208571,0.688571,0.085714,0.231429,0.125714,0.574286,0.311429,0.191429,...,0.014286,0.017143,0.017143,0.017143,0.011429,0.017143,0.017143,0.0,0.0,0.0
std,0.488123,0.468238,0.406869,0.463741,0.280342,0.422349,0.332002,0.495159,0.463741,0.393989,...,0.118836,0.129989,0.129989,0.129989,0.106444,0.129989,0.129989,0.0,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
50%,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
75%,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0


Given the mail is non-spam, found the probabilities of a word existing

In [ ]:
#given Y = 0, prob of word i appearing

wordProbs0 = []
columns = 0
while columns <= 2499:
  sums = trainLabel0_doesWordExist[columns].sum() / len(trainLabel0_doesWordExist.axes[0])
  wordProbs0.append(sums)
  columns += 1

print(wordProbs0)

[0.38857142857142857, 0.32285714285714284, 0.20857142857142857, 0.6885714285714286, 0.08571428571428572, 0.23142857142857143, 0.12571428571428572, 0.5742857142857143, 0.31142857142857144, 0.19142857142857142, 0.35714285714285715, 0.49714285714285716, 0.21428571428571427, 0.24, 0.07142857142857142, 0.15142857142857144, 0.38, 0.3914285714285714, 0.28285714285714286, 0.017142857142857144, 0.09428571428571429, 0.022857142857142857, 0.09714285714285714, 0.37142857142857144, 0.2057142857142857, 0.34, 0.37142857142857144, 0.04, 0.4857142857142857, 0.08571428571428572, 0.37142857142857144, 0.09142857142857143, 0.18571428571428572, 0.3314285714285714, 0.24857142857142858, 0.15142857142857144, 0.07428571428571429, 0.2857142857142857, 0.13428571428571429, 0.07142857142857142, 0.15428571428571428, 0.28285714285714286, 0.04285714285714286, 0.22857142857142856, 0.35714285714285715, 0.16, 0.20285714285714285, 0.11428571428571428, 0.2857142857142857, 0.32, 0.06, 0.23714285714285716, 0.1685714285714285

For spam mails, we are checking if the word exists or not as well.

In [ ]:
trainLabel1_doesWordExist = trainLabel1
columns = 0
while columns <= 2499:
  rows = 0
  while rows < 350:
    if trainLabel1_doesWordExist.iloc[rows][columns] > 1:
      trainLabel1_doesWordExist.iloc[rows][columns] = 1
    rows +=1
  columns += 1

trainLabel1_doesWordExist.describe()

,0,1,2,3,4,5,6,7,8,9,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,Label
count,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,...,350.000000,350.000000,350.000000,350.0,350.000000,350.000000,350.000000,350.0,350.000000,350.0
mean,0.528571,0.474286,0.371429,0.014286,0.200000,0.511429,0.637143,0.042857,0.440000,0.282857,...,0.005714,0.002857,0.005714,0.0,0.017143,0.005714,0.011429,0.0,0.022857,1.0
std,0.499898,0.500053,0.483878,0.118836,0.400573,0.500585,0.481512,0.202825,0.497098,0.451032,...,0.075485,0.053452,0.075485,0.0,0.129989,0.075485,0.106444,0.0,0.149662,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1.0
50%,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1.0
75%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,0.0,1.000000,1.0


Given the mail is spam, found the probabilities of a word existing

In [ ]:
#given Y = 1, prob of word i appearing

wordProbs1 = []
columns = 0
while columns <= 2499:
  sums = trainLabel1_doesWordExist[columns].sum() / len(trainLabel1_doesWordExist.axes[0])
  wordProbs1.append(sums)
  columns += 1

print(wordProbs1)

[0.5285714285714286, 0.4742857142857143, 0.37142857142857144, 0.014285714285714285, 0.2, 0.5114285714285715, 0.6371428571428571, 0.04285714285714286, 0.44, 0.28285714285714286, 0.48, 0.43714285714285717, 0.4742857142857143, 0.38, 0.5657142857142857, 0.44857142857142857, 0.5371428571428571, 0.44857142857142857, 0.3514285714285714, 0.4, 0.45714285714285713, 0.32571428571428573, 0.3628571428571429, 0.36857142857142855, 0.44571428571428573, 0.37714285714285717, 0.33714285714285713, 0.35428571428571426, 0.0, 0.44, 0.1457142857142857, 0.41714285714285715, 0.26571428571428574, 0.3142857142857143, 0.011428571428571429, 0.2742857142857143, 0.36, 0.3142857142857143, 0.3457142857142857, 0.4142857142857143, 0.4142857142857143, 0.017142857142857144, 0.2914285714285714, 0.3028571428571429, 0.02, 0.3342857142857143, 0.0, 0.41714285714285715, 0.09714285714285714, 0.28, 0.3028571428571429, 0.18, 0.18571428571428572, 0.29714285714285715, 0.42857142857142855, 0.32, 0.28, 0.19142857142857142, 0.3085714285

For each word in test_features dataset, multiplying the probabilites found above with the amount of times that word appeared in test set.

Prob0 stores the probability of non-spam mail given the words

Prob1 stores the probability of spam mail given the words

In [ ]:
import math
prob0 = []
prob1 = []
rows = 0
while rows < 260:
  columns = 0
  sum0 = 0
  sum1 = 0
  while columns <= 2499:
    sum0 += wordProbs0[columns] * test_features.iloc[rows][columns]
    sum1 += wordProbs1[columns] * test_features.iloc[rows][columns]
    columns +=1
  prob0.append(sum0)
  prob1.append(sum1)
  rows += 1

If prob0 is larger that means NB will classify the point as non-spam. Else it will be classified as spam

In [ ]:
predictedLabel = []
for i in range(0,len(prob0)):
  if prob0[i] > prob1[i]:
    predictedLabel.append(1)
  else:
    predictedLabel.append(0)
  i+=1


print(predictedLabel)


[1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
rows=0
truePredicted = 0
falsePredicted = 0

while rows < 260:
  if test_labels.iloc[rows][0] == predictedLabel[rows]:
    truePredicted += 1
  else:
    falsePredicted +=1
  rows += 1


print("True predictions:",truePredicted )
print("False predictions:",falsePredicted )
print("Test Accuracy is:" , truePredicted /(truePredicted+falsePredicted))


True predictions: 23
False predictions: 237
Test Accuracy is: 0.08846153846153847


The accuracy is very low. Out of 260 mails, the classifier managed to get 23 of them correctly.It is only 0.088 of the entire set. So, because of this poor prediction accuracy, it is not a good estimator.

# MAP estimate

In [ ]:
#take sum of all occurences of all words + 1*toplam word sayısı -> payda
#for each word will divide occurences + 1 / payda

#sonra bununla test'i kullanıp prob bul. Ona göre class prediction

train1 = trainDF[trainDF["Label"] == 1]
train0 = trainDF[trainDF["Label"] == 0]

totalApperance0 = []
totalApperance1 = []
columns = 0
while columns <= 2499:
  totalApperance0.append(train0[columns].sum())
  totalApperance1.append(train1[columns].sum())
  columns += 1

print(totalApperance0)
print(totalApperance1)

[321, 252, 122, 1130, 58, 141, 70, 906, 186, 115, 280, 444, 111, 145, 28, 75, 278, 264, 180, 6, 102, 9, 61, 277, 140, 188, 216, 24, 477, 39, 231, 42, 92, 227, 445, 78, 29, 166, 72, 33, 82, 378, 18, 121, 312, 95, 360, 52, 320, 180, 23, 161, 184, 28, 1, 77, 5, 25, 30, 39, 33, 14, 54, 240, 84, 259, 50, 120, 19, 73, 16, 113, 95, 63, 271, 126, 68, 60, 130, 76, 153, 31, 11, 181, 180, 1, 74, 253, 5, 9, 30, 48, 56, 27, 80, 120, 61, 246, 133, 158, 66, 61, 65, 40, 11, 77, 208, 24, 120, 24, 219, 171, 36, 115, 9, 0, 21, 129, 22, 196, 46, 89, 202, 53, 151, 58, 172, 85, 36, 13, 187, 3, 43, 2, 162, 5, 68, 19, 46, 16, 86, 4, 127, 9, 7, 29, 100, 68, 131, 60, 26, 38, 53, 129, 10, 25, 54, 10, 115, 200, 23, 2, 81, 149, 1, 9, 64, 16, 131, 65, 7, 12, 32, 142, 126, 130, 139, 151, 18, 103, 5, 138, 17, 42, 74, 18, 10, 16, 44, 39, 69, 30, 47, 14, 2, 28, 28, 17, 103, 41, 86, 28, 95, 131, 17, 52, 12, 157, 9, 156, 11, 90, 11, 38, 123, 0, 108, 60, 0, 38, 92, 119, 89, 14, 30, 49, 25, 41, 68, 118, 16, 5, 92, 20, 38, 

In [ ]:
estimates0 = []
estimates1 = []

column = 0

while column <= 2499:
  estimates0.append( totalApperance0[column] +1 / sum(totalApperance0)+2500)
  estimates1.append( totalApperance1[column] +1 / sum(totalApperance0)+2500)
  column += 1


TypeError: ignored